# 이미지 인식 - CNN

## MNIST 사례

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

import os
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# seed 값 설정
seed = 0
np.random.seed(seed)
tf.random.set_seed(seed)

In [3]:
# 데이터 불러오기
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()
X_train.shape

(60000, 28, 28)

In [4]:
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1).astype('float32')/255
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1).astype('float32')/255
Y_train = tf.keras.utils.to_categorical(Y_train)
Y_test = tf.keras.utils.to_categorical(Y_test)

In [5]:
# 컨볼루션 신경망 설정
# Conv2D(32 마스크의 갯수 kernel_size 마스크의 크기
#input_shape=(28,28,1) 행,열 컬러(3) 또는 흑백(1)
model = Sequential([
    Conv2D(32, kernel_size=(3,3), input_shape=(28,28,1), activation='relu'),
    Conv2D(64, (3, 3), activation='elu'),
    MaxPooling2D(pool_size=2),
    Dropout(0.25),
    Flatten(),
    Dense(200, activation='elu'),
    Dropout(0.5),
    Dense(100, activation='elu'),  
    Dense(10, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 12, 12, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 9216)              0         
_________________________________________________________________
dense (Dense)                (None, 200)               1843400   
_________________________________________________________________
dropout_1 (Dropout)          (None, 200)               0

In [6]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [7]:
# 모델 최적화 설정
MODEL_DIR = './model/'
if not os.path.exists(MODEL_DIR):
    os.mkdir(MEDEL_DIR)

In [8]:
modelpath = MODEL_DIR + "mnist-cnn-{epoch:02d}-{val_loss:.4f}.hdf5"
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', 
                               verbose=1, save_best_only=True)
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10)

In [ ]:
# 모델의 실행
history = model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=30,
                    batch_size=200, #epochs=5, verbose=2, 
                    callbacks=[early_stopping_callback, checkpointer])

Train on 60000 samples, validate on 10000 samples
Epoch 1/30
59800/60000 [============================>.] - ETA: 0s - loss: 0.2048 - accuracy: 0.9376
Epoch 00001: val_loss improved from inf to 0.05966, saving model to ./model/mnist-cnn-01-0.0597.hdf5
60000/60000 [==============================] - 45s 756us/sample - loss: 0.2045 - accuracy: 0.9377 - val_loss: 0.0597 - val_accuracy: 0.9812
Epoch 2/30
59800/60000 [============================>.] - ETA: 0s - loss: 0.0722 - accuracy: 0.9777
Epoch 00002: val_loss improved from 0.05966 to 0.04045, saving model to ./model/mnist-cnn-02-0.0405.hdf5
60000/60000 [==============================] - 44s 731us/sample - loss: 0.0721 - accuracy: 0.9777 - val_loss: 0.0405 - val_accuracy: 0.9862
Epoch 3/30
59800/60000 [============================>.] - ETA: 0s - loss: 0.0582 - accuracy: 0.9819
Epoch 00003: val_loss did not improve from 0.04045
60000/60000 [==============================] - 77s 1ms/sample - loss: 0.0582 - accuracy: 0.9819 - val_loss: 0.044

In [ ]:
del model

In [ ]:
from tensorflow.keras.models import load_model
model = load_model('model/mnist-cnn-11-0.0355.hdf5')

In [ ]:
# 베스트 모델의 결과 출력 
model.evaluate(X_test, Y_test, verbose=0)[1]

In [ ]:
# 테스트셋의 오차
y_vloss = history.history['val_loss']

# 학습셋의 오차
y_loss = history.history['loss']

In [ ]:
# 그래프로 표현
x_len = np.arange(1, len(y_loss)+1)
plt.figure(figsize=(8,6))
plt.plot(x_len, y_vloss, marker='.', c="red", label='Testset_loss')
plt.plot(x_len, y_loss, marker='.', c="blue", label='Trainset_loss')

# 그래프에 그리드를 주고 레이블을 표시
plt.legend(loc='upper right')
# plt.axis([0, 20, 0, 0.35])
plt.grid()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()